# VFD φ-Spaced Phase Windows Analysis

This notebook reproduces the key results from the technical report:

**"A Technical Evaluation of φ-Spaced Phase Windows Against Lifespan Neuroscience Data"**

VFD Institute Technical Report

## 1. Imports and Setup

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from windows_and_coverage import (
    build_phase_windows,
    load_empirical_ages,
    summarise_coverage_table,
    coverage_for_ages,
    PHI,
    ANCHOR_A,
    W0,
    G,
)

from monte_carlo_baselines import random_age_baseline, random_window_baseline
from ratio_scan import scan_ratios, plot_ratio_scan

## 2. Phase Window Definitions

The VFD model predicts phase centres at $t_k = A \cdot \varphi^k$ for $k = 1, \ldots, 5$,
with breathing windows of half-width $w_k = w_0 \cdot g^{k-1}$.

In [ ]:
windows = build_phase_windows()

print(f"Model parameters:")
print(f"  Anchor A = {ANCHOR_A} years")
print(f"  Golden ratio φ = {PHI:.6f}")
print(f"  Base half-width w₀ = {W0} years")
print(f"  Expansion factor g = {G}")
print()

print("Phase Windows:")
print("-" * 60)
for w in windows:
    print(
        f"P{w.index}: centre={w.centre:.2f} yrs, "
        f"window=[{w.lower:.1f}, {w.upper:.1f}], "
        f"half-width={w.half_width:.2f}"
    )

## 3. Empirical Coverage Analysis

Compute coverage of 16 empirical inflection ages extracted from four neuroscience datasets.

In [ ]:
df = load_empirical_ages()
coverage_table = summarise_coverage_table(df, windows)

display(coverage_table[["dataset", "age", "phase", "covered"]])

C_phi = coverage_table["coverage_fraction"].iloc[0]
n_covered = int(C_phi * len(coverage_table))
print(f"\nCoverage C_φ = {C_phi:.4f} ({n_covered} of {len(coverage_table)} ages covered)")

## 4. Random-Age Baseline

Monte Carlo simulation: sample 16 ages uniformly from [0, 90] years,
compute coverage against fixed VFD windows.

In [ ]:
ra = random_age_baseline(n_iter=20_000)
cov = ra["all_coverages"]
p_ge = np.mean(cov >= C_phi)

print("Random-Age Baseline (20,000 iterations)")
print("=" * 45)
print(f"Mean coverage: {ra['mean_coverage']:.4f}")
print(f"Std coverage:  {ra['std_coverage']:.4f}")
print(f"P(C >= C_φ):   {p_ge:.4f}")
print()
print(f"Expected from paper: mean ≈ 0.83, p ≈ 0.22")

## 5. Random-Window Baseline

Monte Carlo simulation: fix empirical ages, fix window widths,
randomise window centres uniformly in [0, 90].

In [ ]:
rw = random_window_baseline(n_iter=20_000)
cov_w = rw["all_coverages"]
p_ge_w = np.mean(cov_w >= C_phi)

print("Random-Window Baseline (20,000 iterations)")
print("=" * 45)
print(f"Mean coverage: {rw['mean_coverage']:.4f}")
print(f"Std coverage:  {rw['std_coverage']:.4f}")
print(f"P(C >= C_φ):   {p_ge_w:.4f}")
print()
print(f"Expected from paper: mean ≈ 0.63, p ≈ 0.055")

## 6. Ratio Scan Analysis

Scan scaling ratios $r \in [1.4, 1.9]$ to evaluate whether φ is uniquely optimal
or sits within a broader effective band.

In [ ]:
results = scan_ratios(r_min=1.4, r_max=1.9, n_points=51)
ratios = results["ratios"]
coverages = results["coverages"]
best_r = results["best_r"]
best_cov = results["best_coverage"]

print("Ratio Scan Results")
print("=" * 45)
print(f"Best ratio: r ≈ {best_r:.4f} with coverage {best_cov:.4f}")
print(f"φ coverage: {C_phi:.4f}")

mask = coverages >= C_phi
n_ge = mask.sum()
if mask.any():
    effective_min = ratios[mask].min()
    effective_max = ratios[mask].max()
    print(f"\nEffective band (C(r) >= C_φ): [{effective_min:.3f}, {effective_max:.3f}]")
    print(f"Number of ratios >= C_φ: {n_ge} of {len(ratios)} ({100*n_ge/len(ratios):.1f}%)")
else:
    print("No ratios achieved coverage >= C_φ in the scanned range.")

In [ ]:
# Plot coverage vs ratio
plt.figure(figsize=(8, 5))
plt.plot(ratios, coverages, marker="o", markersize=4, linewidth=1)
plt.axvline(PHI, color="red", linestyle="--", alpha=0.7, label=f"φ ≈ {PHI:.3f}")
plt.axhline(C_phi, color="green", linestyle=":", alpha=0.5, label=f"C_φ = {C_phi:.2f}")
plt.xlabel("Scaling ratio r", fontsize=11)
plt.ylabel("Coverage C(r)", fontsize=11)
plt.title("Coverage of empirical ages as a function of scaling ratio r", fontsize=12)
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig("coverage_vs_ratio.png", dpi=300)
plt.show()

## 7. Phase Window Visualisation

Visualise the VFD phase windows with empirical inflection ages overlaid.

In [ ]:
df = load_empirical_ages()
ages = df["age"].to_numpy()
datasets = df["dataset"].to_numpy()

fig, ax = plt.subplots(figsize=(10, 3))

# Draw windows as horizontal bands
colors = plt.cm.Blues(np.linspace(0.2, 0.5, len(windows)))
for w, c in zip(windows, colors):
    ax.axvspan(w.lower, w.upper, alpha=0.3, color=c, label=f"P{w.index}")
    ax.axvline(w.centre, linestyle="--", color=c, alpha=0.7)

# Scatter ages with vertical offset by dataset
unique_datasets = list(dict.fromkeys(datasets))  # preserve order
y_pos = {ds: i for i, ds in enumerate(unique_datasets)}
dataset_colors = plt.cm.Set1(np.linspace(0, 1, len(unique_datasets)))

for ds, dc in zip(unique_datasets, dataset_colors):
    mask = datasets == ds
    ax.scatter(ages[mask], [y_pos[ds]] * mask.sum(), s=60, color=dc, 
               edgecolors="black", linewidth=0.5, label=ds, zorder=5)

ax.set_yticks(list(y_pos.values()))
ax.set_yticklabels(unique_datasets)
ax.set_xlabel("Age (years)", fontsize=11)
ax.set_xlim(0, 90)
ax.set_title("VFD Phase Windows with Empirical Inflection Ages", fontsize=12)
ax.legend(loc="upper right", fontsize=8, ncol=2)

plt.tight_layout()
plt.savefig("vfd_phase_windows.png", dpi=300)
plt.show()

## 8. Summary of Key Results

Consolidated results table matching the paper.

In [ ]:
print("Summary of Results")
print("=" * 50)
print(f"VFD coverage (C_φ):                    {C_phi:.4f} ({n_covered}/16)")
print()
print("Random-age baseline:")
print(f"  Mean coverage:                       {ra['mean_coverage']:.4f}")
print(f"  P(C >= C_φ):                         {p_ge:.4f}")
print()
print("Random-window baseline:")
print(f"  Mean coverage:                       {rw['mean_coverage']:.4f}")
print(f"  P(C >= C_φ):                         {p_ge_w:.4f}")
print()
print("Ratio scan:")
print(f"  Best performing ratio:               r ≈ {best_r:.3f}")
print(f"  Effective band (C >= C_φ):           [{effective_min:.3f}, {effective_max:.3f}]")
print()
print("Interpretation:")
print("  - φ performs well but is not uniquely optimal")
print("  - A band of ratios r ≈ 1.5-1.7 achieves comparable coverage")
print("  - This suggests logarithmic-geometric scaling more broadly")
print("    may be the operative principle")